In [54]:
!pip install py-heat-magic z3-solver line_profiler

Defaulting to user installation because normal site-packages is not writeable


In [55]:
from z3 import *
import csv

In [56]:
# basic puzzle
word_length = 5

def define_letter_variables():
    return [z3.Int(f"letter_{index}") for index in range(word_length)]

def add_alphabet_modeling_constraints(solver, letter_vars):
    for letter_var in letter_vars:
        solver.add(letter_var >= 0, letter_var <= 25)

    return solver

letter_to_index_map = {letter: index for index, letter in enumerate("abcdefghijklmnopqrstuvwxyz")}
index_to_letter_map = {index: letter for letter, index in letter_to_index_map.items()}

def pretty_print_solution(model, letter_vars):
    word = []
    for index, letter_var in enumerate(letter_vars):
        word.append(index_to_letter_map[model[letter_var].as_long()])

    word = ''.join(word)
    return word

# dictionary
def load_dictionary():
    dictionary = []
    with open('words.csv', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        dictionary = list(map(lambda x: x[0].strip().lower(), spamreader))
    return dictionary

def add_legal_words_constraints(solver, words, letter_vars):
    all_words_disjunction = []

    for word in words:
        word_conjuction = z3.And([letter_vars[index] == letter_to_index_map[letter] for index, letter in enumerate(word)])
        all_words_disjunction.append(word_conjuction)

    solver.add(z3.Or(all_words_disjunction))

    return solver

# letter frequency
def add_doesnt_contain_letter_constraint(solver, letter_vars, letter):
    for letter_var in letter_vars:
        solver.add(letter_var != letter_to_index_map[letter])

    return solver

def add_contains_letter_constraint(solver, letter_vars, letter):
    solver.add(z3.Or([letter_var == letter_to_index_map[letter] for letter_var in letter_vars]))

    return solver

def add_invalid_position_constraint(solver, letter_vars, letter, position):
    solver.add(letter_vars[position] != letter_to_index_map[letter])

    return solver

def solvingModel(solver, step):
    print("[SOLVING] " + step)
    result = solver.check()
    print(result)

    model = solver.model()
    pretty_print_solution(model, letter_vars)

# subsequent guesses
def add_exact_letter_position_constraint(solver, letter_vars, letter, position):
    solver.add(letter_vars[position] == letter_to_index_map[letter])

    return solver

def add_letter_appears_once_constraint(solver, letter_vars, letter):
    unique_letter_disjunction = []

    for letter_var in letter_vars:
        this_letter_conjunction = [letter_var == letter_to_index_map[letter]]
        for other_letter_var in letter_vars:
            if letter_var == other_letter_var:
                continue
            this_letter_conjunction.append(other_letter_var != letter_to_index_map[letter])
        unique_letter_disjunction.append(z3.And(this_letter_conjunction))

    solver.add(z3.Or(unique_letter_disjunction))

    return solver

# friendlier controlls
def green(solver, letter_vars, letter, position):
    add_exact_letter_position_constraint(solver, letter_vars, letter, position)

def yellow(solver, letter_vars, letter, position):
    add_letter_appears_once_constraint(solver, letter_vars, letter)
    add_invalid_position_constraint(solver, letter_vars, letter, position)

def grey(solver, letter_vars, letter, position):
    add_doesnt_contain_letter_constraint(solver, letter_vars, letter)

def second_grey(solver, letter_vars, letter, position):
    add_invalid_position_constraint(solver, letter_vars, letter, position)



In [62]:
def add_similar_word_to_main(solver, letter_vars, word):
    if len(word) != word_length: raise Exception('the word is not correct')
    word = word.lower()
    equation = []
    for position, letter in enumerate(word):
        equation.append(
            And(letter_vars[position] != letter_to_index_map[letter], 
                Or([letter_var == letter_to_index_map[letter] for letter_var in letter_vars])))

    solver.add(Or(equation))

    return solver

def add_similar_word_to_clue(solver, letter_vars, clue, main):
    if len(clue) != word_length: raise Exception('the word is not correct')
    clue = clue.lower()
    equation = []
    for position, letter in enumerate(clue):
        is_same_position = letter_vars[position] == letter_to_index_map[letter]
        # at least one siminal letter
        equation.append(Or([letter_var == letter_to_index_map[letter] for letter_var in letter_vars]))

        # can't have the same yellow at the same position
        is_green = letter_to_index_map[letter] == letter_to_index_map[main[position]]
        is_yellow = And(Or([letter_to_index_map[letter] == letter_to_index_map[main_letter] for main_letter in main]), # contains word
                    Not(is_green))

        solver.add(Implies(is_yellow, Not(is_same_position)))

        # can't have a green after a grey/yello
        solver.add(
            Implies(Not(is_green), # not a green
                letter_vars[position] != letter_to_index_map[main[position]]))
                
    solver.add(Or(equation))

    # Can't have an unused previous clue
    for letter in main:
        if letter not in clue:
            add_doesnt_contain_letter_constraint(solver, letter_vars, letter)

    return solver



In [58]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [63]:
def find_solution_for_word(word):
    if len(word) != 5: raise Exception('the word is not correct')
    word = word.lower()

    # build basic solver (~60% time)
    solver = Solver()
    letter_vars = define_letter_variables()
    solver = add_alphabet_modeling_constraints(solver, letter_vars)
    words = load_dictionary() 
    solver = add_legal_words_constraints(solver, words, letter_vars)

    # find solution (~40% time)
    solution = [word]
    add_similar_word_to_main(solver, letter_vars, solution[0])
    while solver.check() != unsat:
        # addsimilar word
        # solver.maximize(Sum([If(letter_vars[position] == letter_to_index_map[letter], 1, 0) for position, letter in enumerate(word)]))
    
        model = solver.model()
        similar_word = pretty_print_solution(model, letter_vars)
        solution.append(similar_word)
        add_similar_word_to_clue(solver, letter_vars, similar_word, solution[0])

    return solution


find_solution_for_word("taunt")

['taunt', 'truth', 'thrum']

In [60]:
import random
from multiprocessing import Pool

if __name__ == "__main__":
    words = load_dictionary() 

    
    print(len(words))
    choices = words #random.choices(words, k=100)
    with Pool(8) as p:
        solutions = p.map(find_solution_for_word, choices)
        maxSolution = []
        for solution in solutions:
            if len(maxSolution) < len(solution):
                maxSolution = solution
        print(len(maxSolution))
        print(maxSolution) # ~ 3h

# ['thief', 'often', 'steal', 'duvet', 'clove', 'beard', 'enjoy']
# ['zonal', 'extra', 'stake', 'taffy', 'acrid']
# ['fauna', 'urban', 'ascot', 'beach', 'caddy', 'satyr', 'pizza', 'voila', 'opera', 'maybe'] # eroor opera should block
# ['taunt', 'truth', 'tweed', 'twice', 'tempo', 'tepid', 'tiger', 'thief', 'their', 'title', 'tilde', 'style']
# [12] fix: ['taunt', 'truth', 'title', 'tweed', 'twice', 'tempo', 'tepid', 'tiger', 'thief', 'their', 'tilde', 'style']
# given a word to solve
# find the slowest hard solution

2315


Process ForkPoolWorker-87:
Process ForkPoolWorker-88:
Process ForkPoolWorker-90:
Exception ignored in: 

KeyboardInterrupt: 

In [ ]:
# for each words
# for each letters
# rules
# 1. if a letter from the main isn't used it can never be used
# 2. can't have the same yellow at the same position
# 3. can't have a green after a grey/yello

# heuristics
# - sort candiates by most green, then most yellow  